In [1]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
import pprint
import io
import sys
sys.path.append('../../')


from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor
from architectures import vgg16net, smallnet1, smallnet2
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, train_model, train_log, log_test_score, hp_sweep

from architectures import build_net


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [3]:
""""
grid search
'learning_rate': {
            'values': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]
"""

'"\ngrid search\n\'learning_rate\': {\n            \'values\': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]\n'

In [4]:
config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        'dropout':{
            'values': [0.5, 0.4, 0.3]
        },
        'weight_decay':{
            'values': [1e-5,2e-5, 3e-5,4e-5]
        },
        'epochs':{
            'value': 80
        },
        'lin_layer_size': {
            'values': [100] #, 150, 50
        },
        'first_lin_lay':{
            'values':[554112] #1x554112 and 539904x100
        },
        'optimizer': {
            'values': ['adam']
        },
        'scheduler': {
            'values': [0.2, 0.3, 0.1, 0.5]
        },
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'log_uniform_values',
            'min': 5e-5,
            'max': 1e-4
        },
        'loss_fn': {
            'values': ['MSE', 'CrossEntropy']
        },
        'data_set':{
        'values': ['Augmented']
        },
        'ks': {
            'values': [(3,5), (2, 4), (3,3)]
        },
        'channels': {
            'values': [3]
        },
        'image_path': {
            'values': [r'/its/home/nn268/antvis/optics/AugmentedDS_IDSW/']
        },
        'model_name' : {'values': ['smallnet1']},
        'num_classes' : {
            'values':[11]
        },
    }
}

col_dict = {
    'colour': 'colour',
    'size': [452,144],
    'padding': 5
}

title = f'IDSW_unwrap_4c3l_112023'

save_dict = {'Run' : title,
            'Current_Epoch': 0}


sweep_id = wandb.sweep(config, project=f"112023_4conv3lin_IDSW_aug_{col_dict['colour']}_{col_dict['size']}")

Create sweep with ID: b1pnahyw
Sweep URL: https://wandb.ai/antvis/112023_4conv3lin_IDSW_aug_colour_%5B452%2C%20144%5D/sweeps/b1pnahyw


In [5]:
"""
config = dict(
    epochs= 80, 
    learning_rate =3.0779e-5,
    dataset= 'IDSW',
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    dropout = 0.4,
    first_lin_lay =4096,
    lin_layer_size= 100,
    ks = 3,
    in_chan = 3
)



col_dict = {
    'colour': 'colour',
    'size': [32,32]
}"""

"\nconfig = dict(\n    epochs= 80, \n    learning_rate =3.0779e-5,\n    dataset= 'IDSW',\n    architecture ='CNN',\n    optimizer= 'adam',\n    weight_decay= 4e-5,\n    dropout = 0.4,\n    first_lin_lay =4096,\n    lin_layer_size= 100,\n    ks = 3,\n    in_chan = 3\n)\n\n\n\ncol_dict = {\n    'colour': 'colour',\n    'size': [32,32]\n}"

In [6]:
pprint.pprint(config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'channels': {'values': [3]},
                'data_set': {'values': ['Augmented']},
                'dropout': {'values': [0.5, 0.4, 0.3]},
                'epochs': {'value': 80},
                'first_lin_lay': {'values': [554112]},
                'image_path': {'values': ['/its/home/nn268/antvis/optics/AugmentedDS_IDSW/']},
                'ks': {'values': [(3, 5), (2, 4), (3, 3)]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.0001,
                                  'min': 5e-05},
                'lin_layer_size': {'values': [100]},
                'loss_fn': {'values': ['MSE', 'CrossEntropy']},
                'model_name': {'values': ['smallnet1']},
                'num_classes': {'values': [11]},
                'optimizer': {'values': ['adam']},
                'scheduler': {'values': [0.2, 0.3, 0.1, 0.5]},
            

In [7]:
#model= pipeline(config, col_dict, device)

In [ ]:
def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model = hp_sweep(config, col_dict, save_dict)
wandb.agent(sweep_id, tr, count=20)

wandb: Agent Starting Run: 7vnvzr8t with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	dropout: 0.3
wandb: 	epochs: 80
wandb: 	first_lin_lay: 554112
wandb: 	image_path: /its/home/nn268/antvis/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 3]
wandb: 	learning_rate: 5.1692767458858846e-05
wandb: 	lin_layer_size: 100
wandb: 	loss_fn: CrossEntropy
wandb: 	model_name: smallnet1
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.2
wandb: 	weight_decay: 3e-05


/its/home/nn268/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


improvment in metrics. model saved
